In [7]:
import math
import numpy as np
import scipy
import scipy.stats as ss
import sklearn.cluster as skc
import matplotlib.pyplot as plt
import pandas
import time
from time import mktime
from datetime import datetime

#suppresses pandas indexing warning for cleaner output
#don't worry, I know what I'm doing :)
pandas.options.mode.chained_assignment = None  # default='warn'

admissions = pandas.read_csv('data/ADMISSIONS.csv')
patients = pandas.read_csv('data/PATIENTS.csv')
diagnoses_icd = pandas.read_csv('data/DIAGNOSES_ICD.csv')
d_icd_diagnoses = pandas.read_csv('data/D_ICD_DIAGNOSES.csv')
services = pandas.read_csv('data/SERVICES.csv')

In [11]:
data = pandas.merge(admissions,patients,on='SUBJECT_ID',how='left')
data = pandas.merge(data,services,on='HADM_ID',how='left')
data.drop('SUBJECT_ID_y', axis=1, inplace=True)
data.drop('PREV_SERVICE', axis=1, inplace=True)

In [12]:
data

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,ROW_ID,TRANSFERTIME,CURR_SERVICE
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,...,19,F,2131-05-07 00:00:00,NaN,NaN,NaN,0,24.0,2196-04-09 12:27:00,MED
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,20,M,2082-07-17 00:00:00,NaN,NaN,NaN,0,25.0,2153-09-03 00:21:37,CSURG
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,20,M,2082-07-17 00:00:00,NaN,NaN,NaN,0,26.0,2157-10-18 19:34:59,NSURG
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,...,21,M,2100-05-31 00:00:00,NaN,NaN,NaN,0,27.0,2139-06-06 16:15:36,CMED
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,22,M,2101-11-21 00:00:00,NaN,NaN,NaN,0,28.0,2160-11-02 02:07:20,CMED
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,...,23,M,2054-05-04 00:00:00,2128-02-25 00:00:00,NaN,2128-02-25 00:00:00,1,29.0,2126-05-06 15:18:12,CMED
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,24,F,2191-11-30 00:00:00,NaN,NaN,NaN,0,30.0,2191-11-30 22:21:15,NB
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,25,M,2103-04-15 00:00:00,NaN,NaN,NaN,0,31.0,2177-09-01 00:20:03,CSURG
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,26,M,1872-10-14 00:00:00,NaN,NaN,NaN,0,32.0,2172-10-14 14:18:00,CMED
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,...,27,M,2036-05-17 00:00:00,2108-08-30 00:00:00,2108-08-30 00:00:00,2108-08-30 00:00:00,1,33.0,2108-08-22 23:28:42,MED


In [13]:
data.to_csv('concatenated_data.csv', encoding='utf-8')